# Projet n°2 -

Elyes KHALFALLAH - 5230635

(Tips du prof pour un bon CV : maitriser Spark et Hadoop)

---


In [1]:
import time
from collections import Counter
import re
from multiprocessing import Pool, cpu_count
from collections import defaultdict

# file_path = "large-txt.txt"
file_path = "small-txt.txt"


Avant de commencer, il est important de préprocesser le texte. Pour cela, preprocess_text prend en paramètre le chemin du fichier texte et retourne le texte sans symboles, et en minuscule.


In [2]:
def preprocess_text(file_path):
    with open(file_path, "r") as file:
        text = file.read()

    # Enlève tout symbole qui n'est pas une lettre ou un chiffre et met tout en minuscule
    text = re.sub(r"[^A-Za-z0-9\s]", "", text).lower()

    # Remplace les espaces multiples par un seul espace
    text = re.sub(r"\s+", " ", text)

    return text


preprocessed_text = preprocess_text(file_path)

# Print the first 500 characters of the preprocessed text
if file_path == "small-txt.txt":
    print(preprocessed_text[:500])


jurgen annevelink modern database systems object sql a language for the design and implementation of object databases 1995 rafiul ahad modern database systems object sql a language for the design and implementation of object databases 1995 amelia carlson modern database systems object sql a language for the design and implementation of object databases 1995 daniel h fishman modern database systems object sql a language for the design and implementation of object databases 1995 michael l heytens 


Maintenant, nous allons compter les itérations de chaque mot dans le texte préprocessé, tout en comptant le temps prit pour faire cela

Nous ferons cela en deux parties : une premiere codée à la main, et une deuxième en utilisant une bibliothèque nous permettant de compter en moins de lignes de code (histoire de comparer nos résultats)


In [3]:
# Fonction de compte écrite sans utiliser Counter
def count_word_occurrences(given_text):
    # Démarrer le minuteur
    start_time = time.time()

    # Diviser le texte en mots
    words = given_text.split()
    word_count = {}

    # Compter le nombre d'occurrences de chaque mot
    for word in words:
        if word in word_count:
            word_count[word] += 1
        else:
            word_count[word] = 1

    # Arrêter le minuteur et calculer le temps écoulé
    end_time = time.time()
    elapsed_time = end_time - start_time

    return word_count, elapsed_time


word_count, elapsed_time_no_counter = count_word_occurrences(preprocessed_text)

print(
    f"Temps pris: {elapsed_time_no_counter:.6f} s, soit {elapsed_time_no_counter*1000:.6f} ms"
)

if file_path == "small-txt.txt":
    print(f"\nNombre d'occurrences des mots :\n{word_count}")


Temps pris: 0.001000 s, soit 1.000166 ms

Nombre d'occurrences des mots :
{'jurgen': 2, 'annevelink': 2, 'modern': 66, 'database': 94, 'systems': 141, 'object': 22, 'sql': 8, 'a': 79, 'language': 15, 'for': 103, 'the': 443, 'design': 70, 'and': 265, 'implementation': 10, 'of': 97, 'databases': 150, '1995': 66, 'rafiul': 1, 'ahad': 1, 'amelia': 1, 'carlson': 1, 'daniel': 2, 'h': 8, 'fishman': 2, 'michael': 17, 'l': 5, 'heytens': 1, 'william': 7, 'kent': 4, 'jos': 4, 'blakeley': 2, 'oqlc': 1, 'extending': 1, 'c': 8, 'with': 7, 'an': 22, 'query': 18, 'capability': 1, 'yuri': 2, 'breitbart': 2, 'transaction': 3, 'management': 32, 'in': 57, 'multidatabase': 14, 'hector': 2, 'garciamolina': 2, 'abraham': 1, 'silberschatz': 1, 'stavros': 1, 'christodoulakis': 1, 'multimedia': 4, 'information': 221, 'issues': 4, 'approaches': 2, 'leonidas': 1, 'koveos': 1, 'umeshwar': 2, 'dayal': 2, 'active': 5, 'eric': 2, 'n': 4, 'hanson': 1, 'jennifer': 1, 'widom': 1, 'angelika': 1, 'kotz': 1, 'dittrich': 2,

In [4]:
# Fonction de compte écrite en utilisant Counter
def count_word_occurrences_with_counter(preprocessed_text):
    # Démarrer le minuteur
    start_time = time.time()

    # Diviser le texte en mots
    words = preprocessed_text.split()

    # Compter le nombre d'occurrences de chaque mot
    word_count = Counter(words)

    # Arrêter le minuteur et calculer le temps écoulé
    end_time = time.time()
    elapsed_time = end_time - start_time

    return word_count, elapsed_time


word_count, elapsed_time_with_counter = count_word_occurrences_with_counter(
    preprocessed_text
)

print(
    f"Temps pris : {elapsed_time_with_counter:.6f} secondes, soit {elapsed_time_with_counter*1000:.6f} ms"
)

if file_path == "small-txt.txt":
    print(f"\nNombre d'occurrences des mots :\n{word_count}")


Temps pris : 0.000000 secondes, soit 0.000000 ms

Nombre d'occurrences des mots :
Counter({'the': 443, 'handbook': 315, 'and': 265, 'technology': 225, 'information': 221, 'industrial': 220, '2005': 212, 'databases': 150, 'systems': 141, 'objectoriented': 107, 'for': 103, 'of': 97, 'database': 94, 'applications': 86, 'a': 79, 'design': 70, 'concepts': 69, 'temporal': 68, '1989': 68, 'computer': 68, 'engineering': 67, 'modern': 66, '1995': 66, 'science': 66, '1997': 66, 'in': 57, 'compiler': 43, '1993': 42, 'data': 38, 'system': 37, 'overview': 37, '2002': 37, 'management': 32, 'to': 28, 'automation': 24, 'languages': 23, 'object': 22, 'an': 22, 'distributed': 21, 'on': 20, 'networks': 19, 'query': 18, 'michael': 17, 'dbms': 17, 'architecture': 17, 'introduction': 17, 'control': 17, 'network': 17, 'j': 16, 'model': 16, 'iris': 16, 'language': 15, 'programming': 15, 'manufacturing': 15, 'embedded': 15, 'multidatabase': 14, 'software': 14, 'wireless': 14, 'david': 13, 'processing': 13, 'in

Comparons maintenant les deux outils utilisés :

On voit que les deux méthodes prennent presque exactement le meme temps assez systématiquement, nous n'avons donc pas de choix particulier à faire sur l'outil prendre.

---

Codons maintenant notre Map, et Reduce functions


In [5]:
# Fonction de mapping : cette fonction prend un texte prétraité et produit une liste de paires clé-valeur.
# Chaque paire clé-valeur est un tuple où la clé est un mot unique et la valeur est 1.
def map_function(preprocessed_text):
    words = preprocessed_text.split()
    key_value_pairs = [(word, 1) for word in words]
    print(len(preprocessed_text))
    return key_value_pairs


map_data = map_function(preprocessed_text)

if file_path == "small-txt.txt":
    print(map_data)


52057
[('jurgen', 1), ('annevelink', 1), ('modern', 1), ('database', 1), ('systems', 1), ('object', 1), ('sql', 1), ('a', 1), ('language', 1), ('for', 1), ('the', 1), ('design', 1), ('and', 1), ('implementation', 1), ('of', 1), ('object', 1), ('databases', 1), ('1995', 1), ('rafiul', 1), ('ahad', 1), ('modern', 1), ('database', 1), ('systems', 1), ('object', 1), ('sql', 1), ('a', 1), ('language', 1), ('for', 1), ('the', 1), ('design', 1), ('and', 1), ('implementation', 1), ('of', 1), ('object', 1), ('databases', 1), ('1995', 1), ('amelia', 1), ('carlson', 1), ('modern', 1), ('database', 1), ('systems', 1), ('object', 1), ('sql', 1), ('a', 1), ('language', 1), ('for', 1), ('the', 1), ('design', 1), ('and', 1), ('implementation', 1), ('of', 1), ('object', 1), ('databases', 1), ('1995', 1), ('daniel', 1), ('h', 1), ('fishman', 1), ('modern', 1), ('database', 1), ('systems', 1), ('object', 1), ('sql', 1), ('a', 1), ('language', 1), ('for', 1), ('the', 1), ('design', 1), ('and', 1), ('imple

In [6]:
# Fonction de réduction : cette fonction prend une liste de paires clé-valeur et produit un dictionnaire
def reduce_function(mapped_data):
    # Créer un dictionnaire avec des valeurs par défaut à 0
    word_counts = defaultdict(int)

    for word, value in mapped_data:
        word_counts[word] += value  # Additionner les valeurs associées à chaque mot

    return word_counts


reduced_data = reduce_function(map_data)

if file_path == "small-txt.txt":
    print(reduced_data)


defaultdict(<class 'int'>, {'jurgen': 2, 'annevelink': 2, 'modern': 66, 'database': 94, 'systems': 141, 'object': 22, 'sql': 8, 'a': 79, 'language': 15, 'for': 103, 'the': 443, 'design': 70, 'and': 265, 'implementation': 10, 'of': 97, 'databases': 150, '1995': 66, 'rafiul': 1, 'ahad': 1, 'amelia': 1, 'carlson': 1, 'daniel': 2, 'h': 8, 'fishman': 2, 'michael': 17, 'l': 5, 'heytens': 1, 'william': 7, 'kent': 4, 'jos': 4, 'blakeley': 2, 'oqlc': 1, 'extending': 1, 'c': 8, 'with': 7, 'an': 22, 'query': 18, 'capability': 1, 'yuri': 2, 'breitbart': 2, 'transaction': 3, 'management': 32, 'in': 57, 'multidatabase': 14, 'hector': 2, 'garciamolina': 2, 'abraham': 1, 'silberschatz': 1, 'stavros': 1, 'christodoulakis': 1, 'multimedia': 4, 'information': 221, 'issues': 4, 'approaches': 2, 'leonidas': 1, 'koveos': 1, 'umeshwar': 2, 'dayal': 2, 'active': 5, 'eric': 2, 'n': 4, 'hanson': 1, 'jennifer': 1, 'widom': 1, 'angelika': 1, 'kotz': 1, 'dittrich': 2, 'where': 2, 'objectoriented': 107, 'dbmss': 2,

In [7]:
print(f"Nombre de coeurs disponibles : {cpu_count()} (16 sur ma machine)")


Nombre de coeurs disponibles : 16 (16 sur ma machine)


In [8]:
# Fonction principale
def map_reduce_pipeline(text, num_segments_coeurs):
    """
    Pipeline MapReduce :
    1. Divise les données en segments.
    2. Applique la fonction Map à chaque segment en parallèle.
    3. Agrège les résultats avec la fonction Reduce.

    Arguments :
    - text : texte brut à traiter.
    - num_segments_coeurs : nombre de segments à diviser pour le traitement parallèle (aussi le nombre de coeurs a utiliser)

    Retourne :
    - Résultat final sous forme de dictionnaire contenant la fréquence des mots.
    """
    # Diviser les données en segments
    words = text.split()
    segment_size = len(words) // num_segments_coeurs
    segments = [
        " ".join(words[i : i + segment_size])
        for i in range(0, len(words), segment_size)
    ]
    print("test1")

    # Si les derniers mots ne sont pas inclus dans les segments, les ajouter
    if len(words) % num_segments_coeurs != 0:
        segments[-1] += " " + " ".join(words[num_segments_coeurs * segment_size :])
    print("test2")
    # Appliquer la fonction Map à chaque segment en parallèle
    with Pool(processes=num_segments_coeurs) as pool:
        mapped_results = pool.map(map_function, segments)
    print("test3")
    # Combiner tous les résultats de Map dans une seule liste
    combined_mapped_data = [pair for sublist in mapped_results for pair in sublist]
    print("test4")
    # Appliquer la fonction Reduce pour agréger les résultats finaux
    final_result = reduce_function(combined_mapped_data)
    print("test5")
    return final_result


In [22]:
num_segments_coeurs = 3  # Nombre de segments pour le traitement parallèle
result = map_reduce_pipeline(preprocessed_text, num_segments_coeurs)
print(result)


test1
test2
